In [1]:
import os
import sys
from time import time
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import networkx as nx
import random

from keras import layers, models, optimizers
from keras import backend as K
K.set_image_data_format('channels_last')
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks


from utils import combine_images
from utils import plot_log

from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
sys.path.append('./PatchyTools/')
from GraphConverter import GraphConverter
from DropboxLoader import DropboxLoader
from CapsuleParameters import CapsuleParameters
from CapsuleParameters import CapsuleTrainingParameters
from GraphClassifier import GraphClassifier


import matplotlib.pyplot as plt
%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Load data

In [2]:
# Getting the data:
dataset_name = 'MUTAG'
width = 18
receptive_field = 10
relabeling = True

In [3]:
# Getting the labels:
dropbox_loader = DropboxLoader(dataset_name)
# Importing graph features:
df_edge_label = dropbox_loader.get_edge_label()
df_graph_ind = dropbox_loader.get_graph_ind()
df_adj = dropbox_loader.get_adj()
df_node_label = dropbox_loader.get_node_label()
df_node_label = pd.concat([df_node_label, df_graph_ind.graph_ind], axis=1)
# Getting the labels:
dropbox_loader = DropboxLoader(dataset_name)
graph_labels = dropbox_loader.get_graph_label()
graph_labels = np.array(graph_labels.graph_label)

In [4]:
# Adjeceny dict and 
PatchyConverter = GraphConverter(dataset_name, width, receptive_field)
adj_dict = PatchyConverter.create_adj_dict_by_graphId()
adj_coomatrix_by_graphId = PatchyConverter.create_adj_coomatrix_by_graphId(adj_dict)
#cano_label = canonical_labeling(adj_dict, df_node_label, df_adj)

In [5]:
# Importing graph features:
#df_edge_label = dropbox_loader.get_edge_label()
df_graph_ind = dropbox_loader.get_graph_ind()
df_adj = dropbox_loader.get_adj()
df_node_label = dropbox_loader.get_node_label()
df_node_label = pd.concat([df_node_label, df_graph_ind.graph_ind], axis=1)

In [6]:
# Converting Graphs into Matrices:
if relabeling:
    PatchyConverter.relabel_graphs()
graph_tensor = PatchyConverter.graphs_to_Patchy_tensor()

MUTAG tensor exists, loading it from Dropbox


In [7]:
# Capsule Architecture Parameters:
capsule_params= CapsuleParameters()

# First conv layer: 'filters', kernel_size)
conv_layer_name = 'conv_layer'
conv_layer_params = {}
conv_layer_params['filters'] = 256
conv_layer_params['kernel_size'] = 9
conv_layer_params['strides'] = [1, 1]
conv_layer_params['padding'] = 'VALID'
conv_layer_params['activation'] = 'relu'
conv_layer_params['name'] = 'conv1'

capsule_params.add_params(conv_layer_params,conv_layer_name)

# First Capsule Layer:
# [num_output_caps, caps_len,'filters',kernel_size,strides,padding]
caps_layer_name = 'caps_layer'
caps_layer_params = {}
caps_layer_params['filters'] = 256
caps_layer_params['kernel_size'] = 2
caps_layer_params['strides'] = [2,2]
caps_layer_params['padding'] = 'VALID'
caps_layer_params['padding'] = 'VALID'
caps_layer_params['n_channels'] = 32
caps_layer_params['dim_capsule'] = 8
caps_layer_params['name'] = 'caps_layer'
capsule_params.add_params(caps_layer_params,caps_layer_name)

# Digit Capsule Layer:
digit_layer_name = 'digitcaps_layer'
digit_layer_params = {}
digit_layer_params['n_channels'] = 10
digit_layer_params['dim_capsule'] = 16
digit_layer_params['name'] = 'digitcaps'
capsule_params.add_params(digit_layer_params,digit_layer_name )

# Capsule Decoder:
decoder_layer = 'decoder_layer'
decoder_params = {}
decoder_params['first_dense'] = 256 #250 #512
decoder_params['second_dense'] = 512
decoder_params['name'] = 'decoder'
capsule_params.add_params(decoder_params,decoder_layer)


In [8]:
parameter_set = CapsuleTrainingParameters()
parameter_set.add_fold(1)
parameter_set.epochs= 5

In [9]:
x_train, x_test, y_train, y_test = train_test_split(graph_tensor,graph_labels,test_size=0.10,random_state=1)
data = ((x_train, y_train), (x_test, y_test))
input_shape = x_train.shape[1:]

In [13]:
patchy_classifier = GraphClassifier(input_shape)
patchy_classifier.build_the_graph(capsule_params)
patchy_classifier.train(data, parameter_set)

time to generate the model: 0.2570040225982666
Train on 169 samples, validate on 19 samples
Epoch 1/5
169/169 [==============================] - 1s 4ms/step - loss: 0.8464 - capsnet_loss: 0.7967 - decoder_loss: 0.1267 - capsnet_acc: 0.5858 - val_loss: 0.5343 - val_capsnet_loss: 0.4844 - val_decoder_loss: 0.1272 - val_capsnet_acc: 0.6316
Epoch 2/5
169/169 [==============================] - 0s 830us/step - loss: 0.4185 - capsnet_loss: 0.3688 - decoder_loss: 0.1267 - capsnet_acc: 0.6686 - val_loss: 0.2802 - val_capsnet_loss: 0.2303 - val_decoder_loss: 0.1272 - val_capsnet_acc: 0.6316
Epoch 3/5
169/169 [==============================] - 0s 937us/step - loss: 0.2868 - capsnet_loss: 0.2371 - decoder_loss: 0.1267 - capsnet_acc: 0.6686 - val_loss: 0.3131 - val_capsnet_loss: 0.2633 - val_decoder_loss: 0.1272 - val_capsnet_acc: 0.6316
Epoch 4/5
169/169 [==============================] - 0s 901us/step - loss: 0.3055 - capsnet_loss: 0.2558 - decoder_loss: 0.1267 - capsnet_acc: 0.6686 - val_loss: 0

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [24]:
patchy_classifier.results.rename('wegewg',inplace=True)

epoch                  5.000000
capsnet_acc            0.668639
val_capsnet_acc        0.631579
seg                12525.000000
Name: wegewg, dtype: float64

In [25]:
patchy_classifier.results.name